## Подготовка данных для задания

In [1]:
#!pip install pymorphy2

In [3]:
import re

import pandas as pd
import pymorphy2

text_df = pd.read_csv("content_description.csv", sep='\t')
#text_df = pd.read_csv("./data/content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)

# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
29,0,и,7
37,0,который,5
111,0,шерлок,4
77,0,с,4
84,0,сериал,4
75,0,риколетти,3
31,0,история,2
21,0,для,2
49,0,о,2


# 12.3  Домашняя работа

12.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [29]:
import numpy as np

In [19]:
word_count_df['pos_tag'] = [morph.parse(word)[0].tag.POS for word in word_count_df['word'].tolist()]
word_count_df.head()

,doc_id,word,dummy,pos_tag
0,0,1895,1,None
1,0,а,1,CONJ
2,0,абсолютно,1,ADVB
3,0,англия,1,NOUN
4,0,безобразный,1,ADJF


In [22]:
word_count_df = word_count_df[~word_count_df['pos_tag'].isin(['PREP', 'PRCL'])]
word_count_df.head()

,doc_id,word,dummy,pos_tag
0,0,1895,1,None
1,0,а,1,CONJ
2,0,абсолютно,1,ADVB
3,0,англия,1,NOUN
4,0,безобразный,1,ADJF


In [28]:
all_words = word_count_df['word'].unique()
all_words.shape

(614,)

In [51]:
values = []

for word in all_words:
    row = [word]
    doc_ids = word_count_df['doc_id'].unique()

    count = 0
    for doc_id in doc_ids:
        if word in word_count_df[word_count_df['doc_id'] == doc_id]['word'].to_list():
            count += 1

    row.append(count)
    values.append(row)

doc_frequency = pd.DataFrame(np.array(values),
                             columns=['word', 'count'])
doc_frequency['count'] = doc_frequency['count'].astype(int)
doc_frequency.head()

,word,count
0,1895,1
1,а,4
2,абсолютно,1
3,англия,1
4,безобразный,1


In [58]:
doc_frequency['percentage'] = (doc_frequency['count'] / word_count_df['doc_id'].unique().shape[0]).round(2) * 100
doc_frequency.drop(columns=['count'], inplace=True)
doc_frequency.sort_values('percentage', ascending=False, inplace=True)
doc_frequency.head(10)

,word,percentage
24,и,100.0
98,что,75.0
45,онлайн,75.0
44,он,75.0
69,свой,75.0
15,год,62.0
74,смотреть,62.0
102,этот,50.0
31,который,50.0
126,как,50.0


12.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [65]:
raw_text = text_df['description'].values[3]

# запишите необходимое регулярное выражение в переменную reg_expr 
reg_expr = r'[А-Я]\w+ [А-Я]\w+'

# скомпилированное выражение добавьте в обьёект reg_expr_compiled
reg_expr_compiled = re.compile(reg_expr)

# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

Стивена Фрая
Джоном Дженксом
Теда Уоллеса
Теперь Тед
Вскоре Тед


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`, сохранить в переменнуб `genres_df`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [116]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

values = []
for key in genre_dict.keys():
    for word in genre_dict[key]:
        values.append([word, key])
genres_df = pd.DataFrame(values, columns=['word', 'genre'])
genres_df

,word,genre
0,сатирический,комедия
1,авантюрный,комедия
2,забавный,комедия
3,выбор,мелодрама
4,позор,мелодрама
5,приключения,сказка
6,милый,сказка
7,семейный,сказка
8,тайна,детектив
9,разгадать,детектив


In [117]:
doc_genre = genres_df.merge(word_count_df, on='word', how='inner')
doc_genre

,word,genre,doc_id,dummy,pos_tag
0,сатирический,комедия,3,1,ADJF
1,авантюрный,комедия,4,1,ADJF
2,забавный,комедия,4,1,ADJF
3,выбор,мелодрама,2,1,NOUN
4,позор,мелодрама,2,1,NOUN
5,милый,сказка,1,1,ADJF
6,тайна,детектив,0,1,NOUN
7,разгадать,детектив,0,1,INFN
8,загадочный,детектив,0,1,ADJF
9,ужас,триллер,0,1,NOUN


In [118]:
doc_genre = doc_genre.groupby('doc_id').apply(lambda x: x['genre'].unique()).reset_index()
doc_genre.columns = [doc_genre.columns[0]] + ['genres']
doc_genre

,doc_id,genres
0,0,"[детектив, триллер]"
1,1,[сказка]
2,2,[мелодрама]
3,3,[комедия]
4,4,[комедия]
5,6,[триллер]


In [119]:
for genre in genre_dict.keys():
    doc_genre[genre] = False

for row in range(doc_genre.shape[0]):
    for genre in genre_dict.keys():
        doc_genre.at[row, genre] = genre in doc_genre.loc[row, 'genres']

doc_genre.drop(columns=['genres'], inplace=True)
doc_genre

,doc_id,комедия,мелодрама,сказка,детектив,триллер
0,0,False,False,False,True,True
1,1,False,False,True,False,False
2,2,False,True,False,False,False
3,3,True,False,False,False,False
4,4,True,False,False,False,False
5,6,False,False,False,False,True
